In [2]:
import pandas as pd
import re
import json
import glob
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



In [ ]:
# Define range for file, friends change this help me download when you're free lel
start_int = 0
end_int = 100

total_int = 0
haochen_start_int = 0
haochen_end_int = 100

bernard_start_int = 0
bernard_end_int = 0

yanshiang_start_int = 0
yanshiang_end_int = 0

amanda_start_int = 0
amanda_end_int = 0

tujin_start_int = 0
tujin_end_int = 0
# to update with proper intervals once we know the numbers we want

In [3]:
# Combine CSV, generate output

files = glob.glob("Clean_data/*.csv")

csv = []

for file in files:
    df = pd.read_csv(file, index_col=None, header=0)
    csv.append(df)

df = pd.concat(csv, axis=0, ignore_index=True)

In [4]:
print(df.shape, "\n",df.columns)

(195933, 27) 
 Index(['Unnamed: 0', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'current_currency',
       'deadline', 'fx_rate', 'goal', 'id', 'launched_at', 'location', 'name',
       'pledged', 'profile', 'slug', 'source_url', 'spotlight', 'staff_pick',
       'state', 'state_changed_at', 'static_usd_rate', 'urls', 'usd_pledged',
       'usd_type'],
      dtype='object')


In [5]:
urls = df.urls

urls_ = list(map(lambda url:json.loads(url)["web"]["project"],urls))
urls = urls_

In [56]:
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains

path = "Webdriver/chromedriver.exe"
driver = webdriver.Chrome(path)

updates_all = []

for i in tqdm(range(start_int,end_int)):
    driver.get(urls[i])

    # 1. Load all results
    while True:
        try:
            ActionChains(driver).move_to_element(WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Load more')]")))).pause(5).click().perform()
    #         print("LOAD MORE button clicked")
        except TimeoutException:
    #         print("No more LOAD MORE button to be clicked")
            break

    # 2. Get all updates and save as a list
    updates = []
    try: 
        elements = WebDriverWait(driver,10).until(EC.visibility_of_all_elements_located((By.XPATH,"//article")))
        for element in elements:
            updates.append(element.text)

    except TimeoutException:
        pass
    
    updates_all.append(updates)

driver.quit()

100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [02:46<00:00, 33.37s/it]


In [61]:
df_export = pd.DataFrame({"index":list(range(start_int,end_int)),
                         "update":updates_all}, index = None)
filename = "Update_" + str(start_int) + "-" + str(end_int)
df_export.to_csv("Output/Update/{}.csv".format(filename))